In [1]:

import os
import torch
import random
import numpy as np
import pandas as pd
from os.path import join
from pathlib import Path
from pprint import pprint
from torch.utils.data import DataLoader, TensorDataset

from metaflow import FlowSpec, step, Parameter
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from cleanlab.filter import find_label_issues
from sklearn.model_selection import KFold

from src.system import ReviewDataModule, SentimentClassifierSystem
from src.utils import load_config, to_json
from src.consts import DATA_DIR


/home/gitpod/.pyenv/versions/3.8.13/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config = load_config('config.json')

In [3]:
dm = ReviewDataModule(config)

In [4]:
system = SentimentClassifierSystem(config)

In [5]:
X = np.concatenate([
      np.asarray(dm.train_dataset.embedding),
      np.asarray(dm.dev_dataset.embedding),
      np.asarray(dm.test_dataset.embedding),
    ])
y = np.concatenate([
      np.asarray(dm.train_dataset.data.label),
      np.asarray(dm.dev_dataset.data.label),
      np.asarray(dm.test_dataset.data.label),
    ])

In [14]:
 kf = KFold(n_splits=3)    # create kfold splits

In [9]:
kf.get_n_splits()

3

In [17]:
train_index, test_index = next(kf.split(X))

In [34]:

      X_train = torch.from_numpy(X[train_index]).float()
      y_train = torch.from_numpy(y[train_index]).long()
      X_test = torch.from_numpy(X[test_index]).float()
      y_test = torch.from_numpy(y[test_index]).long()
      train_dataset = TensorDataset(X_train, y_train)
      test_dataset = TensorDataset(X_test, y_test)
      train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
      test_dataloader = DataLoader(test_dataset, shuffle=False)
      
      system = SentimentClassifierSystem(config)

      trainer = Trainer(max_epochs=1)
      trainer.fit(system, train_dataloader)
      test_length = len(test_dataset)
      print(f"test_size: {test_length}")
      probs_ = trainer.predict(model= system, dataloaders=train_dataloader, return_predictions =True)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/gitpod/.pyenv/versions/3.8.13/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:133: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.")

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 49.3 K
-------------------------------------
49.3 K    Trainable params
0         Non-trainable params
49.3 K    Total params
0.197     Total estimated model params size (MB)
/home/gitpod/.pyenv/versions/3.8.13/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of

Epoch 0: 100%|██████████| 1667/1667 [01:43<00:00, 16.04it/s, loss=0.598, v_num=11, train_loss=0.653, train_acc=0.714]
test_size: 26667


/home/gitpod/.pyenv/versions/3.8.13/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:487: PossibleUserWarning: Your `predict_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(
/home/gitpod/.pyenv/versions/3.8.13/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting DataLoader 0: 100%|██████████| 1667/1667 [00:28<?, ?it/s]             


In [44]:
test_dataloader = DataLoader(test_dataset, shuffle=False)
probs_ = trainer.predict(model= system, dataloaders=test_dataloader, return_predictions =True)

/home/gitpod/.pyenv/versions/3.8.13/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting DataLoader 0: 100%|██████████| 26667/26667 [00:52<00:00, 473.71it/s]    


In [45]:
probs_[-1].shape

torch.Size([1, 1])

In [48]:
probs_.numpy()

AttributeError: 'list' object has no attribute 'numpy'

In [50]:
torch.cat(probs_).squeeze().cpu().numpy()

array([0.5916991 , 0.38527182, 0.5396127 , ..., 0.64188683, 0.6187618 ,
       0.45926267], dtype=float32)

In [42]:
y_test.shape

torch.Size([26667])

In [43]:
y_train.shape

torch.Size([53333])

In [29]:
probs_[1]

tensor([[0.3742],
        [0.4174],
        [0.2477],
        [0.4882],
        [0.2129],
        [0.5744],
        [0.4176],
        [0.6219],
        [0.3989],
        [0.2611],
        [0.3277],
        [0.2826],
        [0.5605],
        [0.4761],
        [0.7183],
        [0.4360],
        [0.4635],
        [0.6354],
        [0.6681],
        [0.5378],
        [0.3769],
        [0.7472],
        [0.6324],
        [0.6197],
        [0.3184],
        [0.5591],
        [0.3945],
        [0.5869],
        [0.2113],
        [0.7323],
        [0.4970],
        [0.4588]])

In [31]:
len(test_index)

26667